In [ ]:
import pandas as pd
from minicons import scorer

In [ ]:
# load data
data = pd.read_csv('') # data/data_numbers.csv, data/data_quantifier.csv
df = data.copy()

In [ ]:
# bert_multlingual: 'google-bert/bert-base-multilingual-uncased'
# bert_spanish: 'dccuchile/bert-base-spanish-wwm-uncased'
# roberta_spanish: 'bertin-project/bertin-base-gaussian-exp-512seqlen'

name = ''
model_ckp = ''

In [ ]:
# initialize all models with minicons
model = scorer.MaskedLMScorer(model_ckp, 'cpu')

In [ ]:
def compute_surprisal(text):
    return model.sequence_score(text, reduction=lambda x: -x.mean(0).item())[0]

In [ ]:
df['surprisal_scores'] = df['sentence'].map(compute_surprisal)

In [ ]:
df.to_csv(f'Results/results_{name}', index=False)

### Computing Accuracy

In [ ]:
def compute_accuracy(scores):
   """This function takes a list of surprisal scores as input and calculates the accuracy for three conditions:
       (1) overall accuracy: if algunos_sub < algunos_whole & algunos_whole > unos_whole: 1 else 0,
       (2) only algunos: if algunos_sub < algunos_whole: 1 else 0,
       (3) algunos vs unos: if algunos_whole > unos_whole: 1 else 0."""
   
   scores_len = len(scores)
   
   count_1 = 0
   count_2 = 0
   count_3 = 0
   
   for i in range(0, scores_len, 3):
      item_scores = scores[i: i + 3]
      if item_scores[0] < item_scores[1] and item_scores[1] > item_scores[2]:
         count_1 += 1
      
      if item_scores[0] < item_scores[1]:
         count_2 += 1
      
      if item_scores[1] > item_scores[2]:
         count_3 += 1

   accuracy_1 = round(count_1 / (scores_len / 3), 2)
   accuracy_2 = round(count_2 / (scores_len / 3), 2)
   accuracy_3 = round(count_3 / (scores_len / 3), 2)
   
   return accuracy_1, accuracy_2, accuracy_3

In [ ]:
compute_accuracy(df['surprisal_scores'].tolist())